In [100]:

import json
id_intput = dict()
id_output = dict()
HEAD = "I want you to act as a program developer.\nDo not write any explanations or other words, just reply with the complete function."
lang = 'java'

output_result_file = f'human_eval\\{lang}\\result_java.jsonl'
input_chat_gpt = f'human_eval\\{lang}\\queue_java_api_xai.jsonl'
output_chat_gpt = f'human_eval\\{lang}\\output_queue_java_api_xai.jsonl'

# output_result_file = f'human_eval\\{lang}\\result_{lang}.jsonl'
# input_chat_gpt = f'human_eval\\{lang}\\queue_api_xai_python_humaneval.jsonl'
# output_chat_gpt = f'human_eval\\{lang}\\output_queue_api_xai_python_humaneval.jsonl'


def process_java(source):
    source = source.rstrip()
    if source.endswith('}'):
        source = source[:-1].rstrip()
    if source.endswith('}'):
        source = source[:-1].rstrip()
        
    return source

with open(input_chat_gpt) as f:
    data_input = [json.loads(l.strip()) for l in f.readlines()]
    for obj in data_input:
        id_intput[obj['custom_id']] = obj['body']['messages'][0]['content'].replace(HEAD,'').strip()
with open(output_chat_gpt) as f:
    data = [json.loads(l.strip()) for l in f.readlines()]
    for obj in data:
        source = obj['response']['body']['choices'][0]['message']['content']
        id_output[obj['custom_id']] = source
result = list()
for k,v in id_output.items():
    if lang == 'python':
        result.append({
            'task_id' : k,
            'generation' :  v,
            'prompt' : id_intput[k],
            'function':v
        })
    else:
        # make sure correct code java output
        if '```' in v:
            v = v.replace('```java','')
            v = v.replace('```','')
        if 'public static' in v.splitlines()[0]:
            v = '\n'.join(v.splitlines()[1:])
        # make sure merge test correct         
        vv = process_java(v)
        arrs = id_intput[k].splitlines()
        index = -1
        while 'public static' not in arrs[index]:
            index -= 1
            
        head = '\n'.join(arrs[index:])
        
        func = head +'\n'+ vv + '\n\t}\n}'
        if 'class Problem {' not in func:
            func = 'class Problem {\n'+func
        
        
        result.append({
            'task_id' : k,
            'generation' : id_intput[k] +'\n' + vv,
            'prompt' : id_intput[k],
            'function': func,
        })
with open(output_result_file,'w+') as f:
    for obj in result:
        f.writelines(json.dumps(obj)+'\n')


In [101]:
with open('human_eval\\java\\result_java.jsonl') as f:
    data = [json.loads(l.strip()) for l in f.readlines()]
# for obj in data:
    

In [102]:
print(data[2]['function'])

class Problem {
    public static boolean checkDictCase(HashMap<String,String> dict) {
    if (dict.isEmpty()) {
        return false;
    }
    
    Set<String> keys = dict.keySet();
    boolean allLowerCase = keys.stream().allMatch(key -> key.equals(key.toLowerCase()));
    boolean allUpperCase = keys.stream().allMatch(key -> key.equals(key.toUpperCase()));
    
    return allLowerCase || allUpperCase;
	}
}
